In [1]:
import os
import platform
import sys
from os.path import dirname, realpath

import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# from matplotlib import colors
import matplotlib
matplotlib.use('QtAgg')
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import itertools
import tifffile as tif
import pandas as pd
import seaborn as sns
from pathlib import Path
import glob
from tqdm import tqdm
from scipy import signal
import scipy.io as sio
from scipy.ndimage import gaussian_filter
import imageio
import argparse
from configparser import ConfigParser
import dlc2kinematics
# from pygifsicle import optimize
from get_list_data import expt1_data_list, expt2_data_list
from helper import *
from pylab import *
from matplotlib.path import Path
import matplotlib.patches as patches
from scipy.ndimage import binary_dilation, binary_erosion
import cv2
import mat73

# data_root = '/media/user/teamshare/nick/behavior/grooming/1p/'
data_root = '/media/user/teamshare/pankaj/water_reaching_project/data/matlab_outputs/'
# data_list = expt1_data_list

data_list = ['HR1_2019_4_12data_compile.mat', 'HR2_2019_4_12data_compile.mat', 'IK1_2019_4_12data_compile.mat', 'IK2_2019_4_12data_compile.mat']

# %%

In [2]:
# Define some functions

def tellme(s):
    print(s)
    plt.title(s, fontsize=16)
    plt.draw()

def preprocess_events(data, ke=7, kd=35):
    data = binary_erosion(data, np.ones((ke,)), iterations=1).astype(data.dtype)
    data = binary_dilation(data, np.ones((kd,)), iterations=1).astype(data.dtype)
    return data

def get_start_end(data):
    
    start_idx = np.where(np.diff(data)==1)[0]
    end_idx = np.where(np.diff(data)==255)[0]

    if data[0]:
        start_idx = np.insert(start_idx, 0, 0)

    if data[-1]:
        end_idx = np.insert(end_idx, len(end_idx), len(data))

    assert len(start_idx) == len(end_idx), 'Length of EVENT START and EVENT END do not match.'
    
    return list(zip(start_idx, end_idx))
    

def merge_events(data1, data2):
    return(data1 | data2)

def save_snippets(file_path, full_video, event_idx, blank_space_size=5):
    for i, event in enumerate(event_idx):
        a, z = event
        groom_snippet = full_video[a:z,:,:]

        # groom_snippet = np.ndarray.transpose(full_video[:,:,a:z], (2, 0, 1))
        tif.imwrite(file_path+"_{}_{}.tif".format(a, z), groom_snippet, photometric='minisblack')
    #     if i == 0:
    #         groom_snippets = full_video[:,:,a:z]
    #     else:
    #         groom_snippets = np.dstack((groom_snippets, np.zeros((full_video.shape[0], full_video.shape[1], blank_space_size)), full_video[:,:,a:z]))
    
    # groom_snippets = np.ndarray.transpose(groom_snippets, (2, 0, 1))
    # tif.imwrite(file_path, groom_snippets, photometric='minisblack')


def get_tracks(file):
    df_dlc = pd.read_csv(file, header=0, skiprows=1)            
    fllx = df_dlc['fll'][1:].to_numpy().astype(np.float32)
    flly = df_dlc['fll.1'][1:].to_numpy().astype(np.float32)
    fllp = df_dlc['fll.2'][1:].to_numpy().astype(np.float32)
    
    flrx = df_dlc['flr'][1:].to_numpy().astype(np.float32)
    flry = df_dlc['flr.1'][1:].to_numpy().astype(np.float32)
    flrp = df_dlc['flr.2'][1:].to_numpy().astype(np.float32)
   
    nosex = df_dlc['nose'][1:].to_numpy().astype(np.float32)
    nosey = df_dlc['nose.1'][1:].to_numpy().astype(np.float32)
    nosep = df_dlc['nose.2'][1:].to_numpy().astype(np.float32)

    return [tuple(np.vstack((fllx, flly, fllp))), tuple(np.vstack((flrx, flry, flrp))), tuple(np.vstack((nosex, nosey, nosep)))]


def draw_ROI_on_figure(str_input='Draw polygon on figure', color='r', delete_fig=True):
    while True:
        pts = []
        tellme(str_input)
        pts = np.asarray(plt.ginput(-1, mouse_stop=MouseButton.RIGHT))
            
        ph = plt.fill(pts[:, 0], pts[:, 1], color, lw=2, alpha=0.5)
            
        tellme('Happy? Key click for yes, mouse click for no')
        if plt.waitforbuttonpress():
            break
    
    if delete_fig:
        plt.close()
    verts = [tuple(i) for i in pts]
    return Path(verts)

In [3]:
data_dict = mat73.loadmat(data_root + os.sep + expt)


NameError: name 'expt' is not defined

In [4]:

for expt in tqdm(data_list):
    # if mouse id to process is passed, process only that mouse
    out_dir = data_root + os.sep + expt[:-4] +  os.sep + "outputs"
    set_path(out_dir)

    # mat_contents = sio.loadmat(data_root + os.sep + expt)
    data_dict = mat73.loadmat(data_root + os.sep + expt)

    
    for i in range(len(data_dict['trialResult'])):
        if 

    frame_number = 100
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()

        groom_events_file = data_root + os.sep + mouse_id + os.sep + rec_dir + os.sep + 'grooming_events_roi_unfiltered.mat'
        groom_events_filtered_file = data_root + os.sep + mouse_id + os.sep + rec_dir + os.sep + 'grooming_events_roi_filtered.mat'

        snippets_folder = data_root + os.sep + mouse_id + os.sep + rec_dir + os.sep + 'snippets'
        set_path(snippets_folder)
        
        # df_dlc = pd.read_csv(dlc_file, header=0, skiprows=1)      
        tracks = get_tracks(dlc_file)
        fllx, flly, fllp = tracks[0]
        flrx, flry, flrp = tracks[1]
        nosex, nosey, nosep = tracks[2]
        
        if True:#not os.path.isfile(groom_events_file):    
            matplotlib.use('QtAgg')
            fig = plt.figure()           
            plt.imshow(np.mean(frame, axis=2), cmap='gray')
            plt.plot(fllx, flly, alpha=0.3, label='FL L', linewidth=0.5)
            plt.plot(flrx, flry, alpha=0.3, label='FL R', linewidth=0.5)
            plt.plot(nosex, nosey, alpha=0.3, label='Nose', linewidth=0.5)
            plt.axis('off')

            licking_path = draw_ROI_on_figure(str_input='Draw LICKING roi', color='r', delete_fig=False)
            flr_lick_index = licking_path.contains_points(np.vstack((flrx, flry)).T)
            fll_lick_index = licking_path.contains_points(np.vstack((fllx, flly)).T)
            lick_index = flr_lick_index | fll_lick_index

            grooming_path = draw_ROI_on_figure(str_input='Draw GROOMING roi', color='b', delete_fig=True)
            flr_groom_index = grooming_path.contains_points(np.vstack((flrx, flry)).T)
            fll_groom_index = grooming_path.contains_points(np.vstack((fllx, flly)).T)
            
            # flr_groom_index[lick_index] = 0
            # fll_groom_index[lick_index] = 0
 
            # mdic = {"FL_L": fll_index, "FL_R": flr_index, "vertices": verts}
            # sio.savemat(groom_events_file, mdic)
        else:
            mat_contents = sio.loadmat(groom_events_file)
            flr_index = mat_contents['FL_R'][0]
            fll_index = mat_contents['FL_L'][0]

        # flr_events = preprocess_events(flr_index, ke=9, kd=25)
        # fll_events = preprocess_events(fll_index, ke=9, kd=25)

        flr_lick_index1 = preprocess_events(flr_lick_index, ke=20, kd=20).astype(np.uint8)
        fll_lick_index1 = preprocess_events(fll_lick_index, ke=20, kd=20).astype(np.uint8)
        flr_groom_index1 = preprocess_events(flr_groom_index, ke=9, kd=25).astype(np.uint8)
        fll_groom_index1 = preprocess_events(fll_groom_index, ke=9, kd=25).astype(np.uint8)

        # exclude licking events from grooming events
        # flr_groom_index1[fll_lick_index1 | flr_lick_index1] = 0
        # fll_groom_index1[fll_lick_index1 | flr_lick_index1] = 0

        # t = xt(flr_groom_index1, 90)
        # matplotlib.use('Agg')
        # plt.figure()
        # plt.plot(t, fll_events, alpha=0.5, linewidth=0.5)
        # plt.plot(t, flr_events-1.2, alpha=0.5, linewidth=0.5)
        # plt.title(mouse_id + "_" + rec_dir)
        # plt.xlabel('Time (s)')
        # plt.yticks(ticks=[0.5, -0.7], labels=['FL_L', 'FL_R'])
        # plt.show()
        # pp.savefig()
        # plt.close()

        all_groom_events = get_start_end(merge_events(flr_groom_index1, fll_groom_index1))
        event_type = []
        for event in all_groom_events:
            start_idx, end_idx = event
            if flr_groom_index1[start_idx:end_idx].any() and fll_groom_index1[start_idx:end_idx].any():
                event_type.append('bilateral')
            elif flr_groom_index1[start_idx:end_idx].any() and not fll_groom_index1[start_idx:end_idx].any():
                event_type.append('right')
            elif not flr_groom_index1[start_idx:end_idx].any() and fll_groom_index1[start_idx:end_idx].any():
                event_type.append('left')
        all_groom_events = np.asarray(all_groom_events)

        all_lick_events = get_start_end(merge_events(flr_lick_index1, fll_lick_index1)) # merge all lick events
        # event_type = []
        for event in all_lick_events:
            start_idx, end_idx = event
            event_type.append('lick')
            # if flr_lick_index1[start_idx:end_idx].any() and fll_lick_index1[start_idx:end_idx].any():
            #     event_type.append('lick_biltateral')
            # elif flr_lick_index1[start_idx:end_idx].any() and not fll_lick_index1[start_idx:end_idx].any():
            #     event_type.append('lick_right')
            # elif not flr_lick_index1[start_idx:end_idx].any() and fll_lick_index1[start_idx:end_idx].any():
            #     event_type.append('lick_left')
        all_lick_events = np.asarray(all_lick_events)


        if all_groom_events.size == 0 and all_lick_events.size == 0:
            continue
        elif all_groom_events.size == 0 and all_lick_events.size != 0:
            all_events = all_lick_events
        elif all_groom_events.size != 0 and all_lick_events.size == 0:
            all_events = all_groom_events
        else:
            all_events = np.vstack((all_groom_events, all_lick_events))
        mdic = {"FL_L": fll_groom_index1, "FL_R": flr_groom_index1, "lick": lick_index, "all_events": all_events, "event_type": event_type}
        # sio.savemat(groom_events_filtered_file, mdic)
        
        b_vid = open_cv_read_video(data_root + os.sep + mouse_id + os.sep + rec_dir + os.sep + mouse_id + '_' + rec_dir + '_trim.mp4')
        b_vid = b_vid[:, :, :, 0]

        
        for ee in set(event_type):
            idx = [ee in i for i in event_type]
            save_snippets(snippets_folder + os.sep + ee, 
                         b_vid, all_events[idx])
        del b_vid



  0%|                                                     | 0/4 [01:47<?, ?it/s]


NameError: name 'dhfhfd' is not defined

In [37]:
plt.figure(), plt.plot(data_dict['paw'][0][0][:,0], data_dict['paw'][0][0][:,1]), plt.show()

(<Figure size 640x480 with 1 Axes>,
 None)

In [39]:
np.shape(data_dict['beh'])


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 5 dimensions. The detected shape was (59, 1, 320, 320, 3) + inhomogeneous part.